ConvNext Task1

In [ ]:
import warnings
import pickle
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import timm  # For ConvNeXt models
import numpy as np
from sklearn.metrics import accuracy_score

batch_size = 16

# Load ConvNeXt model
model = timm.create_model('convnext_small', pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])  # Remove last layer classifier head
feature_extractor.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert tensor to PIL image
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert PIL image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

#This is a class defined to create the dataset
class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img = self.transform(img)
        return img

#Lwp classifier class
class LWPClassifier:
    def __init__(self, num_classes=2):
        self.num_classes = num_classes
        self.prototypes = None  # Prototypes will be initialized during training

    def fit(self, X, y):
        # Initialize prototypes as the mean of features for each class
        self.prototypes = np.array([X[y == c].mean(axis=0) for c in range(self.num_classes)])

    def predict(self, X):
        # Compute distances to prototypes
        distances = np.linalg.norm(X[:, np.newaxis] - self.prototypes, axis=2)  # Shape: [num_samples, num_classes]
        return np.argmin(distances, axis=1)  # Assign to the nearest prototype

# This function uses the pre trained model to extract features in batches and return them
def get_features(images, transform):
    dataset = UnlabeledDataset(images, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    feature_extractor.eval()
    features = []

    with torch.no_grad():
        for batch_images in loader:
            batch_images = batch_images.to(device)
            batch_features = feature_extractor(batch_images)
            batch_features = batch_features.mean([2, 3])  # Global Average Pooling
            features.append(batch_features.cpu())

    # Combine features into a single tensor
    features = torch.cat(features)
    return features

all_eval_features = []
all_eval_labels = []
acc_matrix = np.zeros((10, 10)) #To store the accuracies in matrix format
prev_feature = []
prev_labels = []
data_paths = [f"dataset/part_one_dataset/train_data/{i}_train_data.tar.pth" for i in range(1, 11)]
heldout_paths = [f"dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)]

# Main training and evaluation loop
for loop in range(0,10):
    data = torch.load(data_paths[loop])
    heldout = torch.load(heldout_paths[loop])

    train_images = data['data']
    train_labels = data['targets'] if loop == 0 else prev_labels
    eval_images = heldout['data']
    eval_labels = heldout['targets']

    feature_extractor = feature_extractor.to(device)

    train_features = get_features(train_images, transform) if loop == 0 else prev_feature
    print(f"D{loop+1} training features extracted")

    f = LWPClassifier(num_classes=10)
    f.fit(train_features, train_labels)



    eval_features = get_features(eval_images, transform)
    print(f"D{loop+1} eval features extracted")
    all_eval_features.append(eval_features)
    all_eval_labels.append(eval_labels)

    accuracies = []
    for j in range(0,loop):
        preds = f.predict(all_eval_features[j])
        acc = accuracy_score(all_eval_labels[j], preds)
        accuracies.append(acc)
        acc_matrix[loop][j] = acc

    pred_labels = f.predict(eval_features)
    acc = accuracy_score(eval_labels, pred_labels)
    accuracies.append(acc)
    acc_matrix[loop][loop] = acc
    print(f"f{loop+1} Test Accuracies: {accuracies}\n")

    #Save the model f10 in a file to be used in task 2
    if loop == 9:
        with open("lwp_classifier.pkl", "wb") as file:
            pickle.dump(f, file)
        print("Model f10 is saved in lwp_classifier.pkl\n")

    if loop != 9:
        next_data = torch.load(data_paths[loop + 1])
        next_images = next_data['data']
        next_features = get_features(next_images, transform)
        prev_feature = next_features
        prev_labels = f.predict(next_features)

print("Accuracy matrix\n")
print(acc_matrix)

model.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

D1 training features extracted
D1 eval features extracted
f1 Test Accuracies: [0.9316]

D2 training features extracted
D2 eval features extracted
f2 Test Accuracies: [0.9268, 0.93]

D3 training features extracted
D3 eval features extracted
f3 Test Accuracies: [0.924, 0.9256, 0.9304]

D4 training features extracted
D4 eval features extracted
f4 Test Accuracies: [0.924, 0.9252, 0.932, 0.9312]

D5 training features extracted
D5 eval features extracted
f5 Test Accuracies: [0.9228, 0.9284, 0.934, 0.9312, 0.934]

D6 training features extracted
D6 eval features extracted
f6 Test Accuracies: [0.9216, 0.9248, 0.9296, 0.9248, 0.928, 0.9292]

D7 training features extracted
D7 eval features extracted
f7 Test Accuracies: [0.9144, 0.9204, 0.9232, 0.9228, 0.9216, 0.9224, 0.932]

D8 training features extracted
D8 eval features extracted
f8 Test Accuracies: [0.918, 0.9196, 0.9232, 0.9216, 0.9244, 0.926, 0.9312, 0.9208]

D9 training features extracted
D9 eval features extracted
f9 Test Accuracies: [0.92